<div style="text-align:center;">
    <img src="http://www.cs.wm.edu/~rml/images/wm_horizontal_single_line_full_color.png">
    <h1>CSCI 416-01/516-01: Fundamentals of AI and ML, Fall 2025</h1>
    <h1>Linear least-squares and regression</h1>

# Contents 

- [Fitting a line](#Fitting-a-line)
- [Linear models and least squares](#Linear-models-and-least-squares)
- [The normal equations](#The-normal-equations)
- [The California housing dataset](#The-California-housing-dataset)
    - [Visualizing characteristics of our dataset](#Visualizing-characteristics-of-our-dataset)
- [Building a least squares model](#Building-a-least-squares-model)
     - [The normal equations](#The-normal-equations)
- [Training and test sets](#Training-and-test-sets)
- [Regularization](#Regularization)
    - [Ridge regression](#Ridge-regression)
    - [LASSO](#LASSO)

In [ ]:
from IPython.display import Image

<div style="text-align:center;">
    <img src="https://imgs.xkcd.com/comics/curve_fitting.png"/>
</div>

# Fitting a line

The simplest example of regression is fitting a line to a set of data.  Think of the $x$ coordinate as the predictive variable and the $y$ coordinate as the observation.

In this case the model for the data (green squares) is a line:
$$
y(x) = mx + b.
$$

But which line should we choose?  We will choose the $w_{0}, w_{1}$ that minimize a measure of the misfit between observations and model predictions.

In classical least squares, if the data are $(x_{1}, y_{1}), \ldots, (x_{t}, y_{t})$, we want to choose $m, b$ that minimize
$$
\sum_{i=1}^{t} (y_{i} - (mx_{i} + b))^{2}.
$$
The name "least squares" refers to the fact that we are finding the least value of this sum of squares measure of misfit.

Other measures of misfit could be used, e.g.,
$$
\sum_{i=1}^{t} \lvert y_{i} - (bx_{i} + m) \rvert,
$$
but the least squares misfit turns out to yield a particularly tractable optimization problem.

# Linear models and least squares

More generally, suppose we have $x = (x_{1}, \ldots, x_{t}) \in \mathbb{R}^{n}$ and associated values $y_{1}, \ldots, y_{t}$ (we'll assume the $y$ are scalars for simplicity).

Given functions $\phi_{1}(x), \ldots, \phi_{n}(x)$, a model of the form
$$
  w_{1} \phi_{1}(x) + w_{2} \phi_{2}(x) + \cdots + w_{n} \phi_{n}(x)
$$
is called **linear model** because the parameters $w_{i}$ enter the problem linearly.  The model itself may be a nonlinear function of $x$.

Ideally we would have
\begin{align*}
  w_{1} \phi_{1}(x_{1}) + w_{2} \phi_{2}(x_{1}) + \cdots + w_{n} \phi_{n}(x_{1}) &= y_{1} \\
  w_{1} \phi_{1}(x_{2}) + w_{2} \phi_{2}(x_{2}) + \cdots + w_{n} \phi_{n}(x_{2}) &= y_{2} \\
  \vdots &= \vdots \\
  w_{1} \phi_{1}(x_{t}) + w_{2} \phi_{2}(x_{t}) + \cdots + w_{n} \phi_{n}(x_{t}) &= y_{t}. 
\end{align*}
This is a system of linear equations in the $w_{i}$.  If we let $A$ be the $t \times n$ matrix whose $(i,j)$ entry is $\phi_{j}(x_{i})$, $w = (w_{1}, \ldots, w_{n})$, and $y = (y_{1}, \ldots, y_{t})$, we can express this linear system as
$$
Aw = y.
$$
However, if $t > n$ this system may fail to have a solution (there are more equations than unknowns).  On the other hand, if $t < n$ this system still might not have a solution but if it has one, there are infinitely many solutions (more unknowns than equations).

We address these problems by relaxing the requirement that $Aw = y$.  Instead, we require $w$ to minimize 
$$
\sum_{i=1}^{t} (Aw - y)_{i}^{2} = \| Aw - y \|_{2}^{2}.
$$
The quantity $Aw - y$ is called the **residual**.  A minimizer of this function in $w$ is called a **least squares solution**.

# The normal equations

Minimizers of the least squares function have a simple linear algebraic characterization.

**Theorem.**  A vector $w$ is a least squares solution of $Aw = y$ if and only if $A^{T}A w = A^{T} y$.

The linear system $A^{T}A w = A^{T} y$ is called the **normal equations**.

The name "normal equations" comes from the following terminology: a real matrix $N$ is called **normal** if $N^{T}N = NN^{T}$.  In the case of the normal equations the matrix $A^{T}A$ is normal.

**Proof.**  First we show that if $w$ is a least squares solution then it satisfies the normal equations.  Suppose $w$ minimizes 
$F(w) = \| Aw - y \|_{2}^{2}$.  Then, $F(w) \leq F(u)$ for all other $u$.  Choose an arbitrary vector $h$ and define the following function of a single real variable $t$:
\begin{align*}
  \phi(t)
  &= F(w+th) = (w+th)^{T} A^{T}A (w+th) - 2(w+th)^{T}A^{T}y + y^{T}y \\
  &= w^{T} A^{T}A w - 2w^{T}A^{T}y + y^{T}y + (h^{T} A^{T}A h) t^{2} + 2(h^{T}A^{T}(Aw - y))t \\
  &= F(w) + (h^{T} A^{T}A h) t^{2} + 2(h^{T}A^{T}(Aw - y))t.
\end{align*}
Since $\phi(0) = F(w)$, we know that $\phi(0) \leq \phi(t)$ for all other $t$.  Since $0$ is a local minimizer of $\phi$, it follows that $\phi'(0) = 0$.  Since
\begin{align*}
  \phi'(t) &= 2(h^{T} A^{T}A h) t + 2(h^{T}A^{T}(Aw - y)) \\
  \phi'(0) &= 2(h^{T} A^{T} (Aw - y),
\end{align*}
we must have
$$
  h^{T}A^{T}(Aw - y) = 0.
$$
However, $h$ was arbitrary, so this equation must hold for all $h$, so we must have
$$
  A^{T} (Aw - y) = 0,
$$
which is what we wished to show.

Conversely, suppose $y$ satisfies the normal equation: $A^{T}(Aw - y) = 0$.  Let $z$ be another point in $\mathbb{R}^{n}$.  Set $h = z - w$ and as above define 
$$
  \phi(t) = F(w+th) = F(w) + (h^{T} A^{T}A h) t^{2} + 2(h^{T}A^{T}(Aw - y)t.
$$
Since $A^{T}(Aw - y) = 0$, the last term on the right vanishes, leaving us with
$$
  \phi(t) = F(w+th) 
    = F(w) + (h^{T} A^{T}A h) t^{2} 
    = F(w) + \|A h \|_{2}^{2} t^{2} 
    \geq F(w).
$$
Letting $t = 1$ we obtain
$$
  \phi(1) = F(w+(z-w)) = F(z) \geq F(w).
$$
Since $z$ was arbitrary, we see that $F(w) \leq F(z)$ for all $z$, so $w$ is a least squares solution. 👍

An attraction of the normal equations is that if $A^{T}A$ is nonsingular we can solve
$$
  A^{T}A w = A^{T}y
$$
using the $LU$ decomposition (Gaussian elimination) or the Cholesky factorization.  Moreover, if $A \in \mathbb{R}^{m \times n}$, then $A^{T}A \in \mathbb{R}^{n \times n}$.  Since $n$ is the number of terms in our linear model, it is likely not very large and we have a modest size of linear system to solve.

If $A^{T}A$ is ill-conditioned or singular we can use the $QR$ decomposition.  Since we will need to apply $QR$ to the $m \times n$ matrix $A$, it is likely to be considerably more expensive than the normal equation.  On the other hand, $QR$ is numerically extremely stable.

Alternatively, if $A^{T}A$ is ill-conditioned or singular we can apply regularization (see below) or the pseudoinverse.  The pseudoinverse $A^{+}$ will, in the case where $A$ does not have full row rank and least squares solutions are not unique, will yield the least squares solution with the smallest 2-norm.  The pseudoinverse is typically computed from the singular value decomposition (SVD).

# The California housing dataset

We will build a regression model to predict housing prices in California.  Our dataset has 8 features (described below) and a single target of price.

In [ ]:
from sklearn.datasets import fetch_california_housing
ca = fetch_california_housing()
print(ca.DESCR)

## Visualizing characteristics of our dataset

The Python package [seaborn](http://seaborn.pydata.org/) is a statistical visualization package built on top of [matplotlib](https://matplotlib.org).  You will need to install the <code>seaborn</code> module.

First we convert our feature data to a Pandas dataframe:

In [ ]:
import pandas as pd

cols = ca.feature_names
df = pd.DataFrame(ca.data, columns=cols)
df.describe()

In [ ]:
df.head()

Looks like these are decimal degrees for latitude and longitude.  Since California is in the Western hemisphere the longitude will become more negative as we travel west.

In [ ]:
pd.DataFrame(ca.target, columns=["MedPrice"]).describe()

### Pairwise plots

A simple way to gain insight into multivariate data is to plot pairs of the variables.  The seaborn <tt>pairplot()</tt> function does this.

The pairwise plot of a variable with itself is a histogram of that variable's values.

This may take a little time to generate the 64 plots&hellip;

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style='whitegrid', context='notebook')
sns.pairplot(df, height=2.5)
plt.tight_layout()
# plt.savefig('./figures/scatter.png', dpi=300)
plt.show()

### Pairwise correlations

We can also look at the pairwise correlations of variables.

This plot uses Pearson's correlation coefficient $r$.  We have $-1 \leq r \leq 1$.  If $r < 0$ the variables are negatively correlated&ndash;one tends to decrease as the other increases.  If $r > 0$ the variables are positively correlated&ndash;one tends to increase as the other increases.  

In [ ]:
import numpy as np
cm = np.corrcoef(df.values.T)
sns.set(font_scale=1)
sns.set(rc={"figure.figsize":(8, 8)})
hm = sns.heatmap(cm, 
            cbar=True,
            annot=True, 
            square=True,
            fmt='.2f',
            annot_kws={'size': 15},
            yticklabels=cols,
            xticklabels=cols)

plt.tight_layout()
# plt.savefig('./figures/corr_mat.png', dpi=300)
plt.show()

In [ ]:
sns.reset_orig()
%matplotlib inline

# Building a least squares model

We want find a predictor for y, the median house price in each area.  The model will be a weighted combination of the features in our data:
$$
w_{0} + w_{1} x_{1} + \cdots + w_{n} x_{n}.
$$
We seek model parameters $w$ that minimize
$$
\| Aw - y_\textrm{train} \|_{2}^{2},
$$
where $A$ is the training data <code>X_train</code> with a column of ones prepended:

$$
  Aw = \begin{pmatrix}
         1 & x_{1}^{(1)} & \cdots & x_{n}^{(1)} \\
         1 & x_{1}^{(2)} & \cdots & x_{n}^{(2)} \\
         \vdots & \vdots & \vdots & \vdots \\
         1 & x_{1}^{(t)} & \cdots & x_{n}^{(t)}
       \end{pmatrix}
       \begin{pmatrix}
         w_{0} \\ w_{1} \\ \vdots \\ w_{n}
       \end{pmatrix}
     = \begin{pmatrix}
         w_{0} + x_{1}^{(1)}w_{1} + \cdots + x_{n}^{(1)}w_{n} \\
         w_{0} + x_{1}^{(2)}w_{1} + \cdots + x_{n}^{(2)}w_{n} \\
         \vdots \\
         w_{0} + x_{1}^{(t)}w_{1} + \cdots + x_{n}^{(t)}w_{n}
       \end{pmatrix}.
$$      

To make a prediction, we take the test inputs <code>X_test</code>, prepend a column of ones to form a new matrix $\hat{A}$, and then compute $\hat{A} w$.

We'll use our typical 70/30 split for training/test sets:

In [ ]:
from sklearn.model_selection import train_test_split

X = ca.data
y = ca.target

X_train, X_test, y_train, y_test = \
  train_test_split(X, y.reshape(-1,1), test_size=0.3, random_state=0)

## The normal equations

Let's look at the normal operator $A^{T}A$ for the training data:

In [ ]:
N = X_train.T @ X_train
print(N)

Note that while $A$ is $20640 \times 8$, $A^{T}A$ is only $8 \times 8$.

In [ ]:
print(f"2-norm condition number: {np.linalg.cond(N, p=2):.4e}")

The condition number is acceptable if we work in <code>binary64</code> floating-point.  

It is totally unacceptable if we work in <code>binary32</code>.

### Scale the data  

Let's apply the standard scaling (mean 0, variance 1) to the inputs and see the effect on the normal operator.

In [ ]:
from sklearn.preprocessing import StandardScaler
sc_x = StandardScaler()
X_train_std = sc_x.fit_transform(X_train)

Let's look at the condition number for the normal operator for the scaled data:

In [ ]:
N = X_train_std.T @ X_train_std
print(N)
print(N, end="\n\n")
print(f"2-norm condition number: {np.linalg.cond(N, p=2):.4e}")

A much better condition number!

### Instantiate and fit the linear regressor

We use the [LinearRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) class.

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression

use_scaled_data = False
price_scaling = 1

slr = LinearRegression(fit_intercept=False)
if use_scaled_data:
    sc = StandardScaler()
    clf = make_pipeline(sc, slr)
else:
    clf = make_pipeline(slr)

clf.fit(X_train, y_train)
print(clf)

### Make predictions

We will transform the predictions back to the original variables since its easier to understand the units ($100,000).

In [ ]:
y_pred = clf.predict(X_test)
print(y_pred)

# Evaluating the performance of our predictor

Let's look at the errors in our predictions.  Remember that our predictions are in units of $100,000, so we will scale the residual so that we are seeing the values in dollars.

In [ ]:
from math import sqrt

# Compute the residual: the vector of mismatches between prediction and truth.
r = y_pred - y_test
relerr = r / y_test  # Relative error.

In [ ]:
E = np.column_stack((price_scaling * r, price_scaling * np.abs(r), relerr, np.abs(relerr)))
pd.DataFrame(E, columns=("Error", "Absolute error", "Relative error", "Absolute relative error")).describe()

## Looking at the regression model

The model parameters are in the attributes <code>coef_</code> and <code>intercept_</code>.  The intercept term is the constant term in our model.  It gives the expected value for the output when all of the predictor variables are zero if this situation is meaningful (which it is not, for this model).

The relative magnitudes of the coefficients (taking into account the rescaling of the variables) reflects which variables matter most in making the prediction.

In [ ]:
if hasattr(slr, "intercept"):
    print(f"Intercept term: {slr.intercept_[0]:e}")
for coeff, var in zip(slr.coef_[0], ca.feature_names):
    print(f"{coeff:+5.3f} * {var}")

When we use the unscaled data we see that the most important predictors are median income (MedInc) and the average number of bedrooms.  Interesting, as the average number of rooms increases, the model predicts the price to decrease.

As the latitude and longitude increase, the model also predicts the price to decrease.  Can you think of why this might be so?

When looking at the coefficients of our linear model to try to understand the influence of each explanatory variable, you need to keep in mind
* the magnitude of the coefficients,
* the magnitude of the associated explanatory variables,
* the effect of scaling.

In addition, many of the features are themselves correlated.

Finally, let's check that the normal equations are satisfied for our model parameters $w$ and the training data:

In [ ]:
w = np.array(slr.coef_[0])
AtAw = X_train.T @ (X_train @ w)
AtAw.shape = (8,1)
Aty = X_train.T @ y_train
print(AtAw - Aty)

relerr = np.linalg.norm(AtAw - Aty, ord=2)/np.linalg.norm(Aty, ord=2)
print(f"Relative error (2-norm): {relerr}")

As mentioned previously, we can also solve least squares problems via the SVD and the pseudoinverse:

In [ ]:
A_plus = np.linalg.pinv(X_train)
print(A_plus @ y_train)

# Regularization

We can also control overfitting in regression by using **regularization**.  We add a regularization term to the function to the least-squares error function that penalizes large values of the model coefficients.

## Ridge regression

Ridge regression is a regularized variant of ordinary least squares. It computes $w$ by minimizing
$$
\|\; Aw - y \;\|_{2}^{2} + \alpha \|\; w \;\|_{2}^{2},
$$
where $\alpha > 0$.  The 2-norm penalty term keeps the model parameters from becoming large.  The larger the value of $\alpha$, the greater the effect of the penalty term.

The solution to this problem satisfies
$$
  (A^{T}A + \alpha I) w = A^{T} y.
$$
Even if $A^{T}A$ is not invertible the modified matrix $A^{T}A + \alpha I$ will be.

In [ ]:
from sklearn.linear_model import Ridge

ridge = Ridge(alpha=10)
ridge.fit(X_train, y_train)

y_pred = ridge.predict(X_test)
print(ridge.coef_)

In [ ]:
r = y_pred - np.squeeze(y_test)
relerr = r / np.squeeze(y_test)
print(r.shape)
print(y_pred.shape)
print(y_test.shape)

E = np.column_stack((price_scaling * r, price_scaling * np.abs(r), relerr, np.abs(relerr)))
print(E.shape)
pd.DataFrame(E, columns=("Error", "Absolute error", "Relative error", "Absolute relative error")).describe()

## LASSO

LASSO (least absolute shrinkage and selection operator) is a variant of ordinary least squares. It computes $w$ by minimizing
$$
\|\; Aw - y \;\|_{2}^{2} + \alpha \|\; w \;\|_{1}.
$$
Again, the penalty term keeps the model parameters from becoming large.  However, this function is not differentiable everywhere and there is no simple characterization of its minimizer.

The use of the 1-norm (rather than the 2-norm) has the effect of making components of $w$ zero.  This means that some of the input variables are ignored, thereby simplifying the model.  Higher values of the penalty weight $\alpha$ increases the number of variables that are ignored, at the possible cost of increasing the training and prediction error.

In [ ]:
from sklearn.linear_model import Lasso

lasso = Lasso(alpha=0.1)
lasso.fit(X_train, y_train)

print(lasso.coef_)

Observe that some of the predictor coefficients are zero.  This means they do not figure in the predictor, which means our predictor is simpler than the previous one.

In [ ]:
y_pred = lasso.predict(X_test)
y_pred.shape = (6192, 1)

r = y_pred - y_test
relerr = r / y_test
print(y_pred.shape, y_test.shape, relerr.shape)

E = np.column_stack((price_scaling * r, price_scaling * np.abs(r), relerr, np.abs(relerr)))
pd.DataFrame(E, columns=("Error", "Absolute error", "Relative error", "Absolute relative error")).describe()

Once $\alpha$ is sufficiently large all the variables are ignored and the solution is $w = 0$.  This behavior is related to the exact penalization technique in optimization.

In [ ]:
lasso = Lasso(alpha=42)
lasso.fit(X_train, y_train)

y_pred = lasso.predict(X_test)
print(lasso.coef_)